In [1]:
from dataset import myDataset_labelHM
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

/Users/jskaf/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using CUDA')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print('Using MPS')
else :
    device = torch.device('cpu')
    print('Using CPU')

Using MPS


In [4]:
#Parameters
path_csv = 'data/h&mdataset/articles.csv'
path_images = 'data/h&mdataset/images/'
dataset = myDataset_labelHM(path_images, path_csv)
batch_size = 1
num_epochs = 20
learning_rate = 0.00001
num_classes = dataset.get_num_classes()
num_samples = len(dataset)
num_samples

105099

In [5]:
#Split train/validation/test 
train_size = int(0.8 * num_samples)
val_size = int(0.1 * num_samples)
test_size = num_samples - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#Finetuning of a pretrained model resnet50

import torchvision.models as models
from torchvision import transforms

class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.fc = nn.Linear(2048, num_classes)
    
    def forward(self, x):
        x = self.resnet50(x)
        x = self.fc(x)
        return x

model = ResNet50(num_classes)
model.to(device)

#Loss and optimizer
criterion = nn.CrossEntropyLoss()

#Fine tune all layers of the model
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train(model, train_loader, criterion, optimizer, epoch):
    print('Training epoch {}'.format(epoch+1))
    model.train()
    training_loss = 0.0
    for batch in train_loader:
        images, labels,_ = batch
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        training_loss += loss.item()

        
        #Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return training_loss/len(train_loader)

In [ ]:
def evaluate(model, val_loader, epoch, criterion):
    print('Evaluating epoch {}'.format(epoch+1))
    model.eval()
    validation_loss = 0.0
    #Validation and print of accuracy per class
    with torch.no_grad():
        class_correct = list(0. for i in range(num_classes))
        class_total = list(0. for i in range(num_classes))
        for batch in val_loader:
            images, labels, _ = batch
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

        for i in range(num_classes):
            print('Accuracy of %5s : %2d %%' % (
                dataset.get_class(i), 100 * class_correct[i] / class_total[i]))
    return validation_loss/len(val_loader)
    

In [ ]:
#Train the model
train_losses = []
validation_losses = []
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, epoch)
    validation_loss = evaluate(model, val_loader, epoch, criterion)
    train_losses.append(train_loss)
    validation_losses.append(validation_loss)

In [ ]:
#Plot the losses and save them
import matplotlib.pyplot as plt
plt.plot(train_losses, label='Training loss')
plt.plot(validation_losses, label='Validation loss')
plt.legend()
plt.savefig('losses.png')
plt.show()


In [ ]:
#Test the model for each class
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        class_correct = list(0. for i in range(num_classes))
        class_total = list(0. for i in range(num_classes))
        for batch in test_loader:
            images, labels, _ = batch
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

        for i in range(num_classes):
            print('Accuracy of %5s : %2d %%' % (
                dataset.get_class(i), 100 * class_correct[i] / class_total[i]))
            
test(model, test_loader)

In [ ]:
#Save the model
torch.save(model.state_dict(), 'resnet_finetuned.pth')